In [5]:
import os, sys
import numpy as np
import pandas as pd

from txyl_common.biocyc_facade.pgdb import Pgdb, Dat, Traceable
from local.caching import load, save

In [6]:
root = "../../data/txyl_local/biocyc/pgdbs"
pfs = os.listdir(root)
total = len(pfs)
for i, pf in enumerate(pfs):
    print(f"{i+1} of {total}: {pf} {' '*50}", end='\r')
    db = Pgdb(f"{root}/{pf}")
    
    def get_ins_outs(v: dict):
        direction: str = v.get('REACTION-DIRECTION', ['unk'])[0]
        lefts = set(v.get('LEFT', []))
        rights = set(v.get('RIGHT', []))

        left_o = lefts.difference(rights)
        right_o = rights.difference(lefts)
        catalysts = lefts.intersection(rights)
        
        if 'LEFT' not in direction or 'RIGHT' not in direction: # reversible
            all = lefts.union(rights)
            return all, all, catalysts
        else:
            li = direction.index('LEFT')
            ri = direction.index('RIGHT')

            if li < ri:
                return left_o, right_o, catalysts
            else:
                return right_o, left_o, catalysts

    consumption, production, catalyst_use = {}, {}, {}
    def addc(d: dict, v: str):
        d[v] = d.get(v, 0)+1

    for k, v in db.GetDataTable(Dat.REACTIONS).items():
        ins, outs, cats = get_ins_outs(v)
        for lst, ref in zip([ins, outs, cats], [consumption, production, catalyst_use]):
            for mol in lst:
                addc(ref, mol)

    return 


In [11]:
def get_ins_outs(v: dict):
    direction: str = v.get('REACTION-DIRECTION', ['unk'])[0]
    lefts = set(v.get('LEFT', []))
    rights = set(v.get('RIGHT', []))

    left_o = lefts.difference(rights)
    right_o = rights.difference(lefts)
    catalysts = lefts.intersection(rights)
    
    if 'LEFT' not in direction or 'RIGHT' not in direction: # reversible
        all = lefts.union(rights)
        return all, all, catalysts
    else:
        li = direction.index('LEFT')
        ri = direction.index('RIGHT')

        if li < ri:
            return left_o, right_o, catalysts
        else:
            return right_o, left_o, catalysts

consumption, production, catalyst_use = {}, {}, {}
def addc(d: dict, v: str):
    d[v] = d.get(v, 0)+1

for k, v in db.GetDataTable(Dat.REACTIONS).items():
    ins, outs, cats = get_ins_outs(v)
    for lst, ref in zip([ins, outs, cats], [consumption, production, catalyst_use]):
        for mol in lst:
            addc(ref, mol)

In [13]:
catalyst_use

{'PROTON': 22,
 'AMMONIUM': 1,
 'L-LACTATE': 1,
 'MG+2': 1,
 'CO+2': 1,
 'CPD0-2350': 2,
 'Pi': 3,
 'GLUCONATE': 1,
 'SPERMIDINE': 1,
 'PUTRESCINE': 1,
 '1-4-alpha-D-Glucan': 1,
 'Unfolded-Proteins': 1,
 'ACET': 1,
 'F-': 1,
 'RHAMNOSE': 1,
 '3-Hydroxy-Terminated-DNAs': 1,
 'TYPEIV-EFFECTOR-PROTEINS': 1,
 'Long-Chain-Polyphosphate': 2,
 'CPD-763': 1,
 'CPD-17989': 1,
 'THIOL-GROUP': 1,
 'Glutamates': 1,
 'ZN+2': 2,
 'BETAINE': 1,
 'BIOTIN': 1,
 'ssRNAs': 1,
 'Polysulfides': 2,
 'NITRATE': 1,
 '1-5-L-Arabinooligosaccharides': 1,
 'CIT': 1,
 'ENTEROBACTIN': 1,
 'Bases': 1,
 'Deoxy-Ribonucleosides': 1,
 'RNA-Holder': 2,
 'FORMYL-THF-GLU-N': 1,
 'THF-GLU-N': 1,
 'DNA-N': 2,
 'N5-Formyl-THF-Glu-N': 1,
 'METHYLENE-THF-GLU-N': 1,
 'Prolipoprotein-Cysteines': 1,
 'CPD-3': 1,
 'NA+': 1,
 'C6': 1,
 'Soluble-Heteroglycans': 1,
 'CPD-17927': 1,
 'Maltodextrins': 1,
 'Glucans': 1,
 'ARSENATE': 1,
 'Chitodextrins': 1,
 'ECTOINE': 1,
 'SULFATE': 1,
 'CU+2': 1,
 'NICOTINAMIDE_RIBOSE': 1,
 'GLYCEROL': 

In [4]:
assign_methods = set()

for k, v in db.GetDataTable(Dat.ENZYMES).items():
    ak = "BASIS-FOR-ASSIGNMENT"
    assert ak in v
    assign_methods.add(v[ak][0])
    print(v)
    break
    
assign_methods

{'TYPES': ['Enzymatic-Reactions'], 'COMMON-NAME': ['acetyl-CoA carboxylase carboxyltransferase subunit alpha/beta'], 'BASIS-FOR-ASSIGNMENT': [':AUTOMATED-NAME-MATCH'], 'ENZYME': ['DZ96_RS0232580-MONOMER'], 'INSTANCE-NAME-TEMPLATE': ['ENZRXN-*'], 'PHYSIOLOGICALLY-RELEVANT?': ['T'], 'REACTION': ['ACETYL-COA-CARBOXYLTRANSFER-RXN']}


{':AUTOMATED-NAME-MATCH'}

In [5]:
BL = {':INFERRED-TRANSPORT-RXN', ':PATHWAY-HOLE-FILLER'}